In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.meshes import MakeStructured2DMesh

importing NGSolve-6.2.2204


In [2]:
L = 1
alpha = pi/(2*L)
q = cos(alpha*x)*cos(alpha*y)

E  = 1e6
nu = 0.3

t  = Parameter(1e-1)
D  = (E*t**3)/(12*(1-nu**2))
k = 5/6
G = E/(2*(1+nu))
C = k*G*t
w_ex = (1/(4*D*alpha**4)+1/(2*C*alpha**2))*cos(alpha*x)*cos(alpha*y)
grad_w_ex =  CF( (w_ex.Diff(x),w_ex.Diff(y)) )
beta_ex = 1/(4*D*alpha**3)*CF( (sin(alpha*x)*cos(alpha*y),cos(alpha*x)*sin(alpha*y)))
curl_beta_ex = beta_ex[1].Diff(x)-beta_ex[0].Diff(y)
grad_beta_ex =  CF( (beta_ex[0].Diff(x),beta_ex[0].Diff(y), beta_ex[1].Diff(x),beta_ex[1].Diff(y)), dims=(2,2) )

mesh = MakeStructured2DMesh(quads=False, nx=8,ny=8, mapping= lambda x,y : (L*(-1+2*x),L*(-1+2*y)))
Draw(w_ex, mesh)
Draw(beta_ex, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [6]:
def Material(mat):
    return D*((1-nu)*mat+nu*Trace(mat)*Id(2))
def MaterialInv(mat):
    return 1/D*(1/(1-nu)*mat-nu/(1-nu**2)*Trace(mat)*Id(2))
order=2
fesB = HCurl(mesh, order=order-1, dirichlet=".*")
fesS = HDivDiv(mesh, order=order-1, dirichlet=".*")
fesW = H1(mesh, order=order, dirichlet=".*")
fes = fesW*fesB*fesS
(w,beta,sigma), (v,delta,tau) = fes.TnT()
    
n = specialcf.normal(2)
    
a = BilinearForm(fes, symmetric=True)
a += (-InnerProduct(MaterialInv(sigma),tau) \
          + InnerProduct(tau,grad(beta)) + InnerProduct(sigma,grad(delta)) -1e-9*beta*delta)*dx
a += ( -(sigma*n*n)*(delta*n) - (tau*n*n)*(beta*n) )*dx(element_boundary=True)
a += k*G*t*InnerProduct( grad(w)+beta, grad(v)+delta )*dx

f = LinearForm(fes)
f += q*v*dx

gfsol = GridFunction(fes)
gfw, gfbeta, gfsigma = gfsol.components
    
with TaskManager():
        a.Assemble()
        f.Assemble()
        inv = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky")
        gfsol.vec.data = inv * f.vec

Draw(gfw, mesh, "w")
Draw(gfbeta, mesh, "w")

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.22…

BaseWebGuiScene

In [7]:
print ("L2-error:", sqrt(Integrate((gfw - w_ex)**2, mesh)))

L2-error: 5.064344246577689e-07
